# Goal of this notebook

The purpose of this notebook is to assess and structure the quality of the retail product data.

We'll focus on detecting incomplete or incoherent records, and then computing a quality score per produt in order to produce clean and reliable datasets that can be used for BI dashboards in the final project pipeline.

## Dataset context

The dataset comes from Open Food Facts, an open retail product database containing nutritional and categorical information. This type of datasets typically suffers from missing info, inconsistent values..etc because not all products are documented the same way!


Ensuring data quality is a critical prerequisite before any reporting or analytical use.

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
original_df = pd.read_csv("/kaggle/input/world-food-facts/en.openfoodfacts.org.products.tsv", sep="\t", low_memory=False, encoding='utf-8')

In [3]:
original_df.shape

(356027, 163)

In [4]:
original_df.columns.values

array(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity', 'packaging', 'packaging_tags',
       'brands', 'brands_tags', 'categories', 'categories_tags',
       'categories_en', 'origins', 'origins_tags', 'manufacturing_places',
       'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en',
       'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo',
       'cities', 'cities_tags', 'purchase_places', 'stores', 'countries',
       'countries_tags', 'countries_en', 'ingredients_text', 'allergens',
       'allergens_en', 'traces', 'traces_tags', 'traces_en',
       'serving_size', 'no_nutriments', 'additives_n', 'additives',
       'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n',
       'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
   

### Relevant columns:
we select a few key columns to focus on the most important information for analysis
- Product's identity: product_name, brands, categories → so we know what each product is and can group or clean duplicates.
- Nutritional info per 100g: energy_100g, fat_100g, carbohydrates_100g, proteins_100g, salt_100g → to compare products fairly and check for data quality issues.

In [ ]:
selected_columns = [
    'product_name',
    'brands',
    'categories',
    'energy_100g',
    'fat_100g',
    'carbohydrates_100g',
    'proteins_100g',
    'salt_100g'
]

original_df = original_df[selected_columns]

In [ ]:
original_df.info()

In [ ]:
original_df.describe()

In [ ]:
original_df.head()

In [ ]:
original_df['missing_ratio'] = original_df.isna().mean(axis=1)

In [ ]:
original_df['missing_ratio'].describe()

### note:
0 = no missing, 1 = all missing. On average, each product has about 22.5% of its values missing.

## Basic data quality checks:

Real-world product data like this usually has obvious errors that cannot be detected by statistical models alone. Therefore we apply simple business rules to quickly identify incoherent data.

We focus on two basic consistency checks:
- energy values should always be strictly positive.
- the total amount of macronutrients (fat, carbohydrates, proteins) should not exceed 100g per 100g of product (duh!)

In [ ]:
original_df['energy_valid'] =original_df['energy_100g'] > 0

original_df['macro_sum'] = (
   original_df['fat_100g'].fillna(0) +
   original_df['carbohydrates_100g'].fillna(0) +
   original_df['proteins_100g'].fillna(0)
)

original_df['macro_valid'] =original_df['macro_sum'] <= 100

### Now the quality scores
After identifying missing values and some logical rules, we need to summarize this data's quality. The quality score combines dimensions of data quality into one metric that can be easily used in dashboards and reports.

The score combines:
- data completeness,
- validation of basic business consistency rules (energy and macronutrient consistency)

Each component is weighted according to what i believe about its importance.

In [ ]:
original_df['quality_score'] = (
    0.5 * (1 - original_df['missing_ratio']) +
    0.3 * original_df['energy_valid'].astype(int) +
    0.2 * original_df['macro_valid'].astype(int)
)

Now we simply transform the score into business friendly terms!

In [ ]:
def quality_label(score):
    if score >= 0.8:
        return "Good"
    elif score >= 0.6:
        return "Acceptable"
    else:
        return "To fix"

original_df['quality_label'] = original_df['quality_score'].apply(quality_label)

In [ ]:
original_df.to_csv(
    "/kaggle/working/quality_scored_data.csv",
    index=False
)